Este projeto você irá implementar o processamento de consultas. Nela, você utilizará o índice para retornar uma coleção ordenada e avaliação de algumas consultas selecionadas. Para isso, vocês deverão implementar alguns métodos das seguintes classes.

- `IndexPreComputedVals`: Em alguns modelos, há a necessidade de processar alguns valores para que, no momento da execução da consulta, seja retornado de forma mais rápida. Esta classe analisa o índice e armazena informações necessárias para o calculo de cada tipo de modelagem;
- `RankingModel`: Classe abstrata para a criação dos modelos. Ele possui o método `get_ordered_docs` a ser implementado por suas subclasses;
- `BooleanRankingModel` Classe que retorna um resultado de consulta por meio do [modelo booleano](https://docs.google.com/presentation/d/1V62ll_IXRrsp6TYUHjx_T4jIyIc1ZVJYoOSwsxObybE/edit?usp=sharing)
- `VectorRankingModel`: Classe que retorna um resultado de consulta por meio do [modelo vetorial](https://docs.google.com/presentation/d/1jsD1MpLIl08OnWysDhjp7glc4_K0sH9sKUhRqI8lLLo/edit?usp=sharing)
- `QueryRunner`: Classe principal encarregada de obter a consulta e retornar os resultados;


Este trabalho depende do código do indice (pacote `index`). Assim, você deve adicioná-o apropriadamente para dar continuidade ao projeto. 


## Modelagem Booleana e Vetorial

** Atividade 1 - Modelagem Booleana**: A abordagem booleana neste trabalho é simplificada. O modelo recebe possui o atributo `operator` que é uma instancia do [Enum](https://docs.python.org/3.4/library/enum.html) Operator. Caso o operador seja AND, será feito a operacao de interseção entre todos os documentos contidos ocorrencias de palavras, caso contrario, sendo OR, será feito a união. Exemplo: 

In [ ]:
from index.structure import TermOccurrence
map_ocorrencias = {"saturno":[TermOccurrence(1,1,1),
                            TermOccurrence(3,1,1)],
                     "plutao":[TermOccurrence(2,5,1),
                               TermOccurrence(4,5,1)],
                        "terra":[TermOccurrence(1,2,1),
                            TermOccurrence(2,2,1),
                            TermOccurrence(4,2,1),],
                        "venus":[TermOccurrence(1,3,1),
                                TermOccurrence(2,3,1),
                                TermOccurrence(3,3,1),
                                TermOccurrence(4,3,1)],
                        "marte":[TermOccurrence(1,4,2),
                            TermOccurrence(3,4,1),
                            TermOccurrence(4,4,1),],

                        "mercurio":[TermOccurrence(3,6,1)]          
    }

A intereseção entre `saturno` e `venus` resultará nos documentos 1 e 3 e, a união, nos documentos 1, 2, 3 e 4.

Esta é uma forma bem simplificada para implementarmos o modelo booleano. Para isso, você deverá implementar os métodos `union_all` e `intersection_all` presentes na classe `BooleanRankingModel` no arquivo `ranking_models.py`. Esses métodos recebem como parâmetro um mapa com a lista de correncias de cada termo (similar a exemplificada acima).

In [ ]:
!python3 -m query.tests.ranking_models RankingModelTest.test_boolean_model

**Atividade 2 - TF-IDF: ** Agora, no mesmo arquivo, iremos finalizar a implementação do modelo Vetorial utilizando a classe `VectorRankingModel`. Nesta classe, primeiramente, você deverá implementar os [métodos estáticos](https://daniel-hasan.github.io/cefet-web-grad/classes/python2/) `tf`, `idf` e `tf_idf`. Sendo que $TF = 1+log_2(f_{ij})$ e $IDF_i = log_2(\frac{N}{n_i})$ em que $f_{ij}$ é a frequência do termo $i$ no documento $j$, $N$ é o número de documentos e $n_i$ é o número de documentos que ocorrem o termo $i$. Abaixo, faça testes destes métodos: 

In [ ]:
from query.ranking_models import *
from index.structure import *

index = FileIndex()
precomp = IndexPreComputedVals(index)
vectorRank = VectorRankingModel(precomp)

doc_count=4
freq_term=1
num_docs_with_term=3

tf_esperado = 1
idf_esperado = 0.415
tf_idf_esperado = 0.415

tf_calc = vectorRank.tf(freq_term)
idf_calc = vectorRank.idf(doc_count,num_docs_with_term)

if tf_esperado == tf_calc:
    print('OK')
else:
    print(f'Resultado inesperado para TF: {tf_calc}. Esperava-se: {tf_esperado}')
    

if idf_esperado == idf_calc:
    print('OK')
else:
    print(f'Resultado inesperado para IDF: {idf_calc}. Esperava-se: {idf_esperado}')

tf_idf_calc = vectorRank.tf_idf(doc_count, freq_term, num_docs_with_term)
if tf_idf_esperado == tf_idf_calc:
    print('OK')
else:
    print(f'Resultado inesperado para TF_IDF: {tf_idf_calc}. Esperava-se: {tf_idf_esperado}') 


**Atividade 3 - PreComputedVals: ** No modelo vetorial temos que calcular a norma de cada documento $d_j$. Esse calculo não pode ser feito durante o preprocessamento da consulta. Assim, na classe `IndexPreComputedVals` possui o atributo `document_norm` que é um dicionário que mapeia cada documnto $j$ à sua norma. Esse calculo é feito apenas uma vez ao iniciar o programa. 

Desta forma, você deverá terminar de implementar o método `precompute_vals` que percorre todo o índice e armazena a norma de cada documento.

In [ ]:
!python3 -m query.tests.ranking_models RankingModelTest.test_precomputed_vals

**Atividade 4 - Método `get_ordered_docs` da classe `VectorRankingModel`: ** Usando os métodos implementados anteriormente você deverá ordenar os documentos contidos no mapa de ocorrencias `docs_occur_per_term` de acordo com a consulta `query` utilizando o modelo vetorial. O parametro `query` mapeia um termo presente na consulta, para a sua ocorrencia (objeto da classe `TermOcurrence`. Para cada termo $t$ que ocorre na consulta, `docs_occur_per_term` mapeia cada termo com a lista de ocorrencias dele no índice. Veja abaixo um exemplo destes parametros usando a consulta `to be or not to be`.  Veja que, na consulta, `doc_id = None`.

In [ ]:
from index.structure import *
query = {"to":TermOccurrence(None, 1, 2),
         "be":TermOccurrence(None, 2, 2),
          "or":TermOccurrence(None, 3, 1),
          "not":TermOccurrence(None, 4, 1),}
docs_occur_per_term = {
                                    "to":[TermOccurrence(1, 1, 4),
                                          TermOccurrence(2, 1, 1),],
                                      "be":[TermOccurrence(1, 2, 1),TermOccurrence(2, 2, 1)],
                                     "or":[TermOccurrence(2, 3, 1)],
                                    "not":[TermOccurrence(3, 4, 1)],
                                }

Esse método retorna dois valores: (a) uma lista de ids de documentos ordenada de acordo com o modelo vetorial - use o método e um dicionário que mapeia, para cada documento, o seu peso. O método `rank_document_ids` será útil. 

In [ ]:
!python3 -m query.tests.ranking_models RankingModelTest.test_vector_model

## Processamento da Consulta e Avaliação

Agora você irá fazer o processamento da consulta, informada pelo usuário, além de sua avaliação. A implementação do processamento de consultas será feito na classe `QueryRunner` do arquivo `processing.py`.

**Requisito antes de começar:** o código que foi feito da indexação deve estar funcionando. Será utilizado a base de dados da Wikipédia. Você não deverá fazer a indexação toda quando iniciar o programa, ao invés disso, você deve persistir o indice todo em arquivo após a indexação. Usando FileIndex, como as ocorrencias já estão armazenadas em arquivo, você precisa armazenar apenas o conteúdo do `dic_index`. A [biblioteca json](https://docs.python.org/3/library/json.html) pode ajudar. Este índice será lido do arquivo apenas uma vez no início da execução do programa.

**Criação da coleção de referência: ** Para o projeto realizaremos uma avaliação bem simples, com o único intuito de simularmos um processo real de avaliação. Para tanto, consideraremos como conjunto de consultas de teste apenas três consultas:
'Irlanda'
'Belo Horizonte' e 
'São Paulo'

O conjunto de documentos de teste compreenderá todas as páginas da base de dados da Wikipédia PT-BR utilizadas no projeto.  Para cada consulta, disponibilizamos um arquivo (na pasta `relevant_docs`) com o id de documentos relevantes (separados por vírgula) para as consultas teste.

Por exemplo, um documento $D$ será considerado relevante para a consulta 'Belo Horizonte' somente 
se o id de $D$ estiver no arquivo `belo_horizonte.dat`. Você irá armazenar o conteúdo desses arquivos em memória para diminuir o tempo de busca. Feito isso, a coleção de referência para as três consultas estará montada e pode-se realizar os cálculos de avaliação corretamente.

**Como um artigo foi considerado relevante para uma determinada consulta?** A Wikipedia organiza seus artigos em diversas categorias. Assim, para considerarmos se um artigo da Wikipédia é relevante, utilizamos essas categorias. Assim, para os documentos relevantes para a consulta 'Irlanda' (Arquivo `irlanda.dat`), foram considerados relevantes artigos da seguintes categorias: 
 
- Irlanda
- Economia da Irlanda
- História da Irlanda
- Cultura da Irlanda
- Romancistas da Irlanda
- Físicos da Irlanda
- Reis da Irlanda
- Lordes da Irlanda

Categorias relevantes para a consulta 'Belo Horizonte' (Arquivo `belo_horizonte.dat`): 

- Bairros de Belo Horizonte
- Bandas de Belo Horizonte
- Belo Horizonte
- Edifícios de Belo Horizonte
- Metrô de Belo Horizonte
- Naturais de Belo Horizonte
- Prefeitos de Belo Horizonte
- Vereadores de Belo Horizonte

Categorias relevantes para a consulta 'São Paulo' (arquivo `sao_paulo.dat`):

- Atrações turísticas da cidade de São Paulo
- Áreas protegidas de São Paulo
- Prefeitos de São Paulo
- São Paulo
- Turismo em São Paulo
- Universidades de São Paulo
- Rodovias de São Paulo
- Museus da cidade de São Paulo
- Governadores de São Paulo
- Municípios de São Paulo

**Atividade 5 - Leitura dos arquivos de relevâncias: ** Você deverá implementar o método `get_relevance_per_query` que irá ler todos os arquivo da pasta `relevant_docs` e, com isso, retornar um mapeamento em que a chave é a string de consulta e o valor é o **conjunto** de ids de documentos. Veja um exemplo de retorno com as consultas `Bolívia`, `Brasil` e `Porto Seguro`: 


In [ ]:
retorno = {"Bolívia":{1,3,5,6,233},
            "Brasil":{2,4,5,3},
           "Porto Seguro":{3,43,21,3,12,233}
          }

Faça um teste de execução abaixo: 

**Atividade 6 - count_top_n_relevant: ** Um método que auxilirará vocês na avaliação é o método count_top_n_relevant da classe Query Runner.  Esse método calcula a quantidade de documentos relevantes nas top `n` posições da lista `lstResposta` que é a resposta a uma consulta - lista de ids de documentos. `lstResposta` será a lista de respostas ordenadas por um método de processamento de consulta (BM25, Modelo vetorial). Os ids de documentos relevantes estão no parametro `docRelevantes`.

In [ ]:
!python3 -m query.tests.processing ProcessingTest.test_count_top_n_relevant

**Atividade 7 - processamento da consulta**: O método `get_query_term_occurence` a consulta da mesma forma que foi preprocessado o texto do documento (use a classe `Cleaner` para isso). Este método irá retonar a consulta em um dicionario em que chave é o termo que ocorreu e o valor é uma instancia da classe TermOccurrence (ver Atividade 4). O doc_id deverá ser sempre None. Caso o termo nao exista no indice, ele será desconsiderado.

In [ ]:
!python3 -m query.tests.processing ProcessingTest.test_get_query_term_occurence

**Atividade 8 - Recuperação dos termos da consulta no índice: ** O método `get_occurrence_list_per_term` possui com parametro a lista de termos da consulta. Este método retorna um dicionario com a lista de ocorrencia no indice de cada termo passado como parametro. Caso o termo não exista, este termo possuirá uma lista vazia. Veja o exemplo na atividade 4.

In [ ]:
!python3 -m query.tests.processing ProcessingTest.test_get_occurence_list_per_term

**Atividade 9 - processamento da consulta: ** Este método recebe como parametro a consulta, os valores precomputado do índice e o dicionário de documentos relevantes (extraídos do método `get_relevance_per_query`) para retornar uma lista de IDs ordenados de acordo com a consulta utilizando o modelo de ranking e indice que são os atributos `index` e 

In [ ]:
!python3 -m query.tests.processing ProcessingTest.test_get_docs_term

**Atividade 10: Método estático runQuery e criação da interface de caracteres - ou uso da interface gráfica/web: ** Você deverá fazer um método que utilizando o indice, valores precomputados e o dicionario de indices relevantes:
            
- Instanciar um objeto de uma subclasse de RankingModel, de acordo com o que foi solicitado pelo usuário
- Preprocessar os termos da consulta
- Obter no indice as ocorrencias de cada termo do indice
- Utilize o método get_docs_term para obter a lista de documentos que responde esta consulta
- Caso seja uma consulta que possua documentos relevantes assinalados, fazer a avaliação da precisão e revocação dos top 10, 20 e 50
- Imprimir as top 10 respostas. 
            
Você deverá criar uma interface de carcteres para solicitar ao usuario a consulta e a modelagem a ser usada. Caso opte por uma interface de caracteres,faça também uma função main em processing e, abaixo, execute-a para mostrar como exemplo. Caso deseje, ao inves disso, você pode também fazer uma interface gráfica. Para testar este método, você deverá usar o indice da Wikipedia. Além disso, você deverá fazer uma análise e acordo com o guia de escrita do relatório sobre as as consultas da coleção de referencia. Para melhorar a apresentação, o arquivo `titlePerDoc.dat` apresenta o título do artigo por id do mesmo. O arquivo `itens obrigatorios - reltorio processamento de consulta.pdf` demonstra os itens obrigatórios do relatório.

Você tem a liberdade de alterar este método como bem entender, mas lembre-se que os valores precomputados devem ser executado uma vez só durante a execução do programa antes da solicitação das consultas para que a consulta não fique lenta. Além disso, você deve ler o arquivo do índice também apenas uma vez (não indexe a Wikipedia novamente e sim leia o arquivo do indice gravado em memória). Caso deseje, você pode modificar o IndexPreComputedVals para armazenar mais elementos precomputados que facilitariam a consulta (ou a exibição da mesma). 

 Caso opte por fazer uma interface web, deve estar claro as instruções para que seja possível executa-la, inclusive, as suas dependencias. Para melhoria de organização, a parte de interface grafica ou web deve ser feita em outros arquivos.

Para rodar a api, necessário instalar alguns pacotes:
- pip install flask flask-jsonpify flask-restful
- pip install -U flask-cors

In [ ]:
from flask import Flask, request, jsonify, abort
from flask_restful import Resource, Api
from json import dumps
from flask_cors import CORS, cross_origin

from index.indexer import Cleaner
from query.ranking_models import BooleanRankingModel, VectorRankingModel, IndexPreComputedVals, OPERATOR
from query.processing import QueryRunner
from index.structure import HashIndex,FileIndex,TermOccurrence

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

api = Api(app)

index = FileIndex()
index.index("sao_paulo",35020,1)
index.index("irlanda",39300,3)
index.index("espero",39300,1)
index.index("que",11953,1)
index.index("sao_paulo",16865,2)
index.index("estejam",11953,1)
index.index("se",11953,1)
index.index("sao_paulo",359,5)
index.index("irlanda",37632,4)
index.index("irlanda",11953,1)
index.finish_indexing()

precomp = IndexPreComputedVals(index)

class Buscador(Resource):
    def get(self):
        query = request.args.get('query')
        typeValue = request.args.get('type')
        operation = request.args.get('operation')
        
        if query is None or typeValue is None or operation is None:
            abort(400, "Por favor, envie todos os dados")
        
        print(query, typeValue, operation)
        
        if typeValue == 'booleano':
            if operation == 'and':
                modelo = BooleanRankingModel(OPERATOR.AND)
            else:
                modelo = BooleanRankingModel(OPERATOR.OR)
        else:
            modelo = VectorRankingModel(precomp)
        
        relevantes = QueryRunner.get_relevance_per_query(query) # Recebe todos os docs 
        #queryRunner = QueryRunner(modelo, index, cleaner)
        #print(relevantes)
        QueryRunner.runQuery(query, index, precomp, relevantes, modelo)
        result = [1,2,3,4,5,6]
        return jsonify(result)

api.add_resource(Buscador, '/buscador') 

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-12-05 09:35:31,912] ERROR in app: Exception on /buscador [GET]
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/lib/python3/dist-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/natha/.local/lib/python3.8/site-packages/flask_restful/__init__.py", line 468, in wrapper
    resp = resource(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/flask/views.py", line 89, in view
    return self.dispatch_request(*args, **kwargs)
  File "/home/natha/.local/lib/python3.8/site-packages/flask_restful/__init__.py", line 583, in dispatch_request
    resp = meth(*args, **kwargs)
  File "<ipython-input-1-488a108d2d17>", line 54, in get
    QueryRunner.runQuery(query, index, precomp, relevantes, modelo)
  File "/home/natha/CEFET/RI/c

sao_paulo vetorial and
ola mundou deu bom
1
2
3
Query sao_paulo
Dic_query_occur
{'sao_paulo': (term_id:1 doc: None freq: 1)}
Query dividida
['sao_paulo']
Dic_query_per_term_query
{'sao_paulo': [(term_id:1 doc: 359 freq: 5), (term_id:1 doc: 16865 freq: 2), (term_id:1 doc: 35020 freq: 1)]}
Query
{'sao_paulo': (term_id:1 doc: None freq: 1)}
DCPT
{'sao_paulo': [(term_id:1 doc: 359 freq: 5), (term_id:1 doc: 16865 freq: 2), (term_id:1 doc: 35020 freq: 1)]}
[1, 2, 3, 4, 5, 6]
Documents_weight
{}
Doc IDs
[]
Respostas cheio
([], {})
[]
Passei aqui
{'belo_horizonte': {'130173', '69168', '97000', '108604', '19588', '91593', '49772', '107302', '29241', '1083', '49186', '136473', '38636', '47967', '35985', '484', '106575', '43658', '101043', '91853', '104137', '80198', '40226', '110030', '18111', '125549', '136473\n'}, 'irlanda': {'15393', '124818\n', '3765', '33833', '51778', '95773', '39300', '45577', '105145', '26276', '37632', '44259', '102888', '51716', '43872', '52995', '54836', '51779', '132

In [ ]:
from flask import Flask, request, jsonify, abort
from flask_restful import Resource, Api
from json import dumps
from flask_cors import CORS, cross_origin

from index.indexer import Cleaner
from query.ranking_models import BooleanRankingModel, VectorRankingModel, IndexPreComputedVals, OPERATOR
from query.processing import QueryRunner
from index.structure import HashIndex,FileIndex,TermOccurrence

query = "Ins"
relevantes = QueryRunner.get_relevance_per_query(query)
for teste in relevantes:
    print(teste)
    if ['69168'] in relevantes[teste]:
        print("Entrou")
print(relevantes)




In [ ]:
from index.indexer import Cleaner
from query.ranking_models import BooleanRankingModel, VectorRankingModel, IndexPreComputedVals, OPERATOR
from query.processing import QueryRunner
from index.structure import HashIndex,FileIndex,TermOccurrence

QueryRunner.main()
